<a href="https://colab.research.google.com/github/yamadashamoji/Colab/blob/main/social_game3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

クエストシステム追加

In [1]:
import random
import json
import os

class Character:
    def __init__(self, name, hp, attack):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.attack = attack
        self.level = 1
        self.exp = 0

    def level_up(self):
        self.level += 1
        self.max_hp += 10
        self.hp = self.max_hp
        self.attack += 2
        print(f"{self.name}はレベル{self.level}に上がった！")

class Player(Character):
    def __init__(self, name):
        super().__init__(name, hp=100, attack=10)
        self.items = []
        self.quests = []
        self.completed_quests = []

    def use_item(self, item):
        if item in self.items:
            self.hp = min(self.hp + 20, self.max_hp)
            self.items.remove(item)
            print(f"{self.name}は{item}を使用した。HPが20回復した！")
        else:
            print(f"{item}を持っていません。")

    def add_quest(self, quest):
        self.quests.append(quest)
        print(f"新しいクエスト「{quest.name}」を受注しました！")

    def complete_quest(self, quest):
        if quest in self.quests:
            self.quests.remove(quest)
            self.completed_quests.append(quest)
            print(f"クエスト「{quest.name}」を完了しました！")
            quest.give_reward(self)
        else:
            print("そのクエストは受注していません。")

    def to_dict(self):
        return {
            "name": self.name,
            "hp": self.hp,
            "max_hp": self.max_hp,
            "attack": self.attack,
            "level": self.level,
            "exp": self.exp,
            "items": self.items,
            "quests": [quest.to_dict() for quest in self.quests],
            "completed_quests": [quest.to_dict() for quest in self.completed_quests]
        }

    @classmethod
    def from_dict(cls, data):
        player = cls(data["name"])
        player.hp = data["hp"]
        player.max_hp = data["max_hp"]
        player.attack = data["attack"]
        player.level = data["level"]
        player.exp = data["exp"]
        player.items = data["items"]
        player.quests = [Quest.from_dict(q) for q in data["quests"]]
        player.completed_quests = [Quest.from_dict(q) for q in data["completed_quests"]]
        return player

class Enemy(Character):
    def __init__(self):
        name = random.choice(["スライム", "ゴブリン", "オーク"])
        hp = random.randint(30, 50)
        attack = random.randint(5, 15)
        super().__init__(name, hp, attack)

class Quest:
    def __init__(self, name, description, target_enemy, target_count, reward_exp, reward_item):
        self.name = name
        self.description = description
        self.target_enemy = target_enemy
        self.target_count = target_count
        self.current_count = 0
        self.reward_exp = reward_exp
        self.reward_item = reward_item

    def update_progress(self, enemy):
        if enemy.name == self.target_enemy:
            self.current_count += 1
            print(f"クエスト「{self.name}」: 進捗 {self.current_count}/{self.target_count}")
            if self.is_completed():
                print(f"クエスト「{self.name}」の条件を達成しました！")

    def is_completed(self):
        return self.current_count >= self.target_count

    def give_reward(self, player):
        player.exp += self.reward_exp
        print(f"{self.reward_exp}の経験値を獲得しました！")
        if self.reward_item:
            player.items.append(self.reward_item)
            print(f"{self.reward_item}を獲得しました！")

    def to_dict(self):
        return {
            "name": self.name,
            "description": self.description,
            "target_enemy": self.target_enemy,
            "target_count": self.target_count,
            "current_count": self.current_count,
            "reward_exp": self.reward_exp,
            "reward_item": self.reward_item
        }

    @classmethod
    def from_dict(cls, data):
        quest = cls(
            data["name"],
            data["description"],
            data["target_enemy"],
            data["target_count"],
            data["reward_exp"],
            data["reward_item"]
        )
        quest.current_count = data["current_count"]
        return quest

def battle(player, enemy):
    print(f"{enemy.name}が現れた！")

    while player.hp > 0 and enemy.hp > 0:
        print(f"\n{player.name} HP: {player.hp}/{player.max_hp}")
        print(f"{enemy.name} HP: {enemy.hp}/{enemy.max_hp}")

        action = input("何をする？ (1: 攻撃, 2: アイテム使用): ")

        if action == "1":
            damage = player.attack
            enemy.hp -= damage
            print(f"{player.name}の攻撃！{enemy.name}に{damage}のダメージ！")
        elif action == "2":
            if player.items:
                player.use_item("ポーション")
            else:
                print("アイテムがありません。")
                continue

        if enemy.hp <= 0:
            print(f"{enemy.name}を倒した！")
            exp_gain = random.randint(10, 20)
            player.exp += exp_gain
            print(f"{exp_gain}の経験値を獲得！")
            if player.exp >= 100:
                player.level_up()
                player.exp -= 100

            # クエスト進行状況の更新
            for quest in player.quests:
                quest.update_progress(enemy)
                if quest.is_completed():
                    player.complete_quest(quest)

            return

        damage = enemy.attack
        player.hp -= damage
        print(f"{enemy.name}の攻撃！{player.name}に{damage}のダメージ！")

    if player.hp <= 0:
        print(f"{player.name}は倒れた...")

def save_game(player):
    data = player.to_dict()
    with open("save_data.json", "w") as f:
        json.dump(data, f)
    print("ゲームをセーブしました。")

def load_game():
    if os.path.exists("save_data.json"):
        with open("save_data.json", "r") as f:
            data = json.load(f)
        return Player.from_dict(data)
    return None

def generate_quest():
    enemy_types = ["スライム", "ゴブリン", "オーク"]
    quest_type = random.choice(["討伐", "収集"])
    target_enemy = random.choice(enemy_types)
    target_count = random.randint(3, 7)
    reward_exp = target_count * 10
    reward_item = random.choice(["上級ポーション", "攻撃力上昇の書", "防御力上昇の書"])

    if quest_type == "討伐":
        name = f"{target_enemy}{target_count}体討伐"
        description = f"{target_enemy}を{target_count}体倒す"
    else:
        name = f"{target_enemy}の素材{target_count}個収集"
        description = f"{target_enemy}から素材を{target_count}個集める"

    return Quest(name, description, target_enemy, target_count, reward_exp, reward_item)

def main():
    player = load_game()
    if player:
        print(f"セーブデータを読み込みました。{player.name}としてゲームを再開します。")
    else:
        player_name = input("新しいプレイヤー名を入力してください: ")
        player = Player(player_name)
        player.items = ["ポーション", "ポーション"]

    print(f"冒険を始めます、{player.name}！")

    while player.hp > 0:
        print("\n====== メニュー ======")
        print("1: 戦闘")
        print("2: クエスト確認")
        print("3: 新しいクエストを受注")
        print("4: セーブしてやめる")

        action = input("何をしますか？: ")

        if action == "1":
            enemy = Enemy()
            battle(player, enemy)

            if random.random() < 0.3:  # 30%の確率でアイテムドロップ
                player.items.append("ポーション")
                print("ポーションを見つけた！")

        elif action == "2":
            if player.quests:
                print("\n===== 進行中のクエスト =====")
                for quest in player.quests:
                    print(f"- {quest.name}: {quest.current_count}/{quest.target_count}")
            else:
                print("進行中のクエストはありません。")

        elif action == "3":
            new_quest = generate_quest()
            player.add_quest(new_quest)

        elif action == "4":
            save_game(player)
            print("ゲームを終了します。")
            break

    if player.hp <= 0:
        print("ゲームオーバー")

if __name__ == "__main__":
    main()

新しいプレイヤー名を入力してください: からす
冒険を始めます、からす！

====== メニュー ======
1: 戦闘
2: クエスト確認
3: 新しいクエストを受注
4: セーブしてやめる
何をしますか？: 2
進行中のクエストはありません。

====== メニュー ======
1: 戦闘
2: クエスト確認
3: 新しいクエストを受注
4: セーブしてやめる
何をしますか？: 3
新しいクエスト「スライムの素材7個収集」を受注しました！

====== メニュー ======
1: 戦闘
2: クエスト確認
3: 新しいクエストを受注
4: セーブしてやめる
何をしますか？: 1
スライムが現れた！

からす HP: 100/100
スライム HP: 40/40
何をする？ (1: 攻撃, 2: アイテム使用): 1
からすの攻撃！スライムに10のダメージ！
スライムの攻撃！からすに10のダメージ！

からす HP: 90/100
スライム HP: 30/40
何をする？ (1: 攻撃, 2: アイテム使用): 1
からすの攻撃！スライムに10のダメージ！
スライムの攻撃！からすに10のダメージ！

からす HP: 80/100
スライム HP: 20/40
何をする？ (1: 攻撃, 2: アイテム使用): 1
からすの攻撃！スライムに10のダメージ！
スライムの攻撃！からすに10のダメージ！

からす HP: 70/100
スライム HP: 10/40
何をする？ (1: 攻撃, 2: アイテム使用): 1
からすの攻撃！スライムに10のダメージ！
スライムを倒した！
17の経験値を獲得！
クエスト「スライムの素材7個収集」: 進捗 1/7

====== メニュー ======
1: 戦闘
2: クエスト確認
3: 新しいクエストを受注
4: セーブしてやめる
何をしますか？: 2

===== 進行中のクエスト =====
- スライムの素材7個収集: 1/7

====== メニュー ======
1: 戦闘
2: クエスト確認
3: 新しいクエストを受注
4: セーブしてやめる
何をしますか？: 1
オークが現れた！

からす HP: 70/100
オーク HP: 47/47
何をする？ (1: 攻撃, 2: アイテム使用): 1
からすの